In [1]:
import time
import random
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime
from elasticsearch import Elasticsearch 

import mysql.connector

In [2]:
from elasticsearch.helpers import bulk

# 日期处理

In [3]:
def date_convert(dot_date_str):
    '''xxxx.xx.xx --> xxxx-xx-xx
    then pack into json for es
    '''
    y,m,d = dot_date_str.split('.')
    new_date = '{year}-{month}-{date}'.format(year=y,month=m,date=d)
    return new_date
#     return {new_date:{'type':'date','formate':'yyyy-mm-dd'}}

In [4]:
def date_autofill(dot_date_str):
    '''xxxx --> xxxx-01-01
    then pack into json for es
    '''
    new_date = '{year}-{month}-{date}'.format(year=dot_date_str,month='01',date='01')
    return new_date

# 读取库中paper数据

In [5]:
cnx = mysql.connector.connect(user='root', password='ir@DUT#2018',
                              host='10.8.128.205',port='23306')

cursor = cnx.cursor()

# 获取weibo表头

In [10]:
cursor.execute("select * from information_schema.columns where table_name='weibo_table'")

In [11]:
sql_table_columns = cursor.fetchall()

In [12]:
sql_table_columns_name = [i[3] for i in sql_table_columns]

# 获取weibo表总容量

In [27]:
cursor.execute("select table_rows from information_schema.tables where table_name='weibo_table'")

In [28]:
sql_table_rows = cursor.fetchall()

In [29]:
sql_table_rows = sql_table_rows[0][0]

In [48]:
def data_to_json(data_atom,head_columns):
    data = data_atom
    data_dict = {k:data[num] for num,k in enumerate(head_columns)}
    return data_dict

# 构建索引

In [38]:
es = Elasticsearch(hosts='10.8.128.205',port=49200,timeout=30, max_retries=10, retry_on_timeout=True)

# 每次查询limit数目的数值 同时批量插入paper

In [39]:
# es.indices.delete(index='weibo-tweet-index')

In [40]:
index_name = 'weibo-tweet-index'

In [53]:
limit = 5000
offset = 0
for epoch in range(int(sql_table_rows/limit)):
    print('[{0}] Epoch:{1}'.format(time.ctime(),epoch))
    base_fetch_sql = 'select * from {dbName}.{tableName} limit {limitVolume} offset {offsetVolume}'
    base_fetch_sql = base_fetch_sql.format(dbName='weiboDB',tableName='weibo_table',limitVolume=limit,offsetVolume=offset)
    cursor.execute(base_fetch_sql)
    sql_data = cursor.fetchall()
    sql_dict = [data_to_json(i,sql_table_columns_name) for i in sql_data]
    bulk(es,sql_dict,index=index_name,raise_on_error=True)
    offset += limit - 1
# 最后一桶
base_fetch_sql = 'select * from {dbName}.{tableName} limit {limitVolume} offset {offsetVolume}'
base_fetch_sql = base_fetch_sql.format(dbName='weiboDB',tableName='weibo_table',limitVolume=limit,offsetVolume=offset)
cursor.execute(base_fetch_sql)
sql_data = cursor.fetchall()
sql_dict = [data_to_json(i,sql_table_columns_name) for i in sql_data]
bulk(es,sql_dict,index=index_name,raise_on_error=True)

(3767, [])

In [54]:
cursor.close()

True

# 读取库中comment数据

In [6]:
cnx = mysql.connector.connect(user='root', password='ir@DUT#2018',
                              host='10.8.128.205',port='23306')

cursor = cnx.cursor()

# 获取paper表头

In [7]:
cursor.execute("select * from information_schema.columns where table_name='comment_table'")

In [8]:
sql_table_columns = cursor.fetchall()

In [9]:
sql_table_columns_name = [i[3] for i in sql_table_columns]

In [10]:
sql_table_columns_name

['C_ID',
 'C_comment_id',
 'C_comment_user_id',
 'C_comment_user_nick_name',
 'C_content',
 'C_weibo_url',
 'C_like_num',
 'C_created_at',
 'C_crawl_time']

# 获取comment表总容量

In [11]:
cursor.execute("select table_rows from information_schema.tables where table_name='comment_table'")

In [12]:
sql_table_rows = cursor.fetchall()

In [13]:
sql_table_rows = sql_table_rows[0][0]

# 每次查询limit数目的数值 同时批量插入author

In [63]:
es.indices.delete(index='weibo-comment-index')

{'acknowledged': True}

In [64]:
es = Elasticsearch(hosts='10.8.128.205',port=49200,timeout=30, max_retries=10, retry_on_timeout=True)

In [65]:
index_name = 'weibo-comment-index'

In [66]:
limit = 10000
offset = 0
for epoch in range(int(sql_table_rows/limit)):
    print('[{0}] Epoch:{1}'.format(time.ctime(),epoch))
    base_fetch_sql = 'select * from {dbName}.{tableName} limit {limitVolume} offset {offsetVolume}'
    base_fetch_sql = base_fetch_sql.format(dbName='weiboDB',tableName='comment_table',limitVolume=limit,offsetVolume=offset)
    cursor.execute(base_fetch_sql)
    sql_data = cursor.fetchall()
    sql_dict = [data_to_json(i,sql_table_columns_name) for i in sql_data]
    bulk(es,sql_dict,index=index_name,raise_on_error=True)
    offset += limit - 1
# 最后一桶
base_fetch_sql = 'select * from {dbName}.{tableName} limit {limitVolume} offset {offsetVolume}'
base_fetch_sql = base_fetch_sql.format(dbName='weiboDB',tableName='comment_table',limitVolume=limit,offsetVolume=offset)
cursor.execute(base_fetch_sql)
sql_data = cursor.fetchall()
sql_dict = [data_to_json(i,sql_table_columns_name) for i in sql_data]
bulk(es,sql_dict,index=index_name,raise_on_error=True)

[Fri Jul 19 11:43:30 2019] Epoch:0
[Fri Jul 19 11:43:35 2019] Epoch:1
[Fri Jul 19 11:43:39 2019] Epoch:2
[Fri Jul 19 11:43:43 2019] Epoch:3
[Fri Jul 19 11:43:48 2019] Epoch:4
[Fri Jul 19 11:43:52 2019] Epoch:5
[Fri Jul 19 11:43:57 2019] Epoch:6
[Fri Jul 19 11:44:01 2019] Epoch:7
[Fri Jul 19 11:44:05 2019] Epoch:8
[Fri Jul 19 11:44:10 2019] Epoch:9
[Fri Jul 19 11:44:14 2019] Epoch:10
[Fri Jul 19 11:44:19 2019] Epoch:11


(10000, [])

# 设置ES状态为可写（触发硬盘存储超限后集群会自动配置所有索引为只读）

In [42]:
import json

import requests

headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Content-Type': 'application/json',
    'Origin': 'chrome-extension://ffmkiejjmecolpfloofpjologoblkegm',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

unlock_dict = {
"index.blocks.read_only_allow_delete": "false"
}

ppp = requests.put('http://10.8.128.205:49200/paper-year-index/_settings',data=json.dumps(unlock_dict),headers=headers)

# 论文检索对象